In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [75]:
glcm = pd.read_csv('C:/Users/ravee/Jupyter/Skin-Cancer-Classification/data/glcm3.csv', header=None)
dataset = pd.read_csv('C:/Users/ravee/Jupyter/Skin-Cancer-Classification/data/color3.csv', header=None)
X = dataset.iloc[:, 0:12].values
Y = glcm.iloc[:, 24]

In [76]:
dataset

,0,1,2,3,4,5,6,7,8,9,10,11
0,67.374839,63.340830,112.263065,55.737793,52.463312,91.327983,-0.033850,-0.032123,1.950237,1.981436,481.104062,509.562271
1,52.300242,82.668481,137.457367,49.635884,75.253828,121.745271,0.134248,-0.066341,2.015002,1.921611,344.643799,448.534121
2,76.645767,92.508307,148.996292,57.005206,67.924656,106.590925,0.057193,-0.050644,2.050657,2.006676,1026.553558,546.204784
3,11.615295,17.457194,27.802821,33.447148,48.877430,76.538757,0.077760,-0.670850,2.222315,2.906180,176.992461,304.594632
4,46.209791,62.129607,101.965131,57.775117,75.554092,121.504623,0.146505,0.023257,1.958404,2.095389,369.719816,343.396839
...,...,...,...,...,...,...,...,...,...,...,...,...
916,82.044136,83.768722,120.307772,89.496293,89.498528,126.176035,-0.343964,0.078492,2.098720,2.102761,239.172011,154.305262
917,83.793886,79.859312,125.879575,71.660835,68.321651,105.753942,-0.102729,0.081119,1.886748,2.044726,98.200153,83.670620
918,82.216087,63.332681,107.854577,81.626920,64.118188,105.661451,0.564203,-0.436504,2.575104,2.013750,118.218423,254.623358
919,58.616472,58.697274,100.933170,65.831264,65.248561,108.459665,0.098212,-0.091045,1.973688,1.955592,104.419733,87.910685


In [77]:
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [78]:
len(X_train)

736

In [79]:
len(X_test)

185

In [80]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Naive Bayes

In [8]:
# Implementation of Naive Bayes
# Fitting Naive Bayes to the training set

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [9]:
# Predicting the test set result
y_pred = nb.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[48,  9, 22],
       [ 8, 29, 23],
       [ 5, 16, 25]], dtype=int64)

In [10]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = nb, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.5378378378378379


In [11]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = nb, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.68      0.57      0.62        40
           2       0.54      0.41      0.46        32
           3       0.34      0.57      0.43        21

    accuracy                           0.52        93
   macro avg       0.52      0.52      0.50        93
weighted avg       0.55      0.52      0.52        93

              precision    recall  f1-score   support

           1       0.76      0.56      0.65        39
           2       0.52      0.45      0.48        33
           3       0.21      0.35      0.26        20

    accuracy                           0.48        92
   macro avg       0.49      0.46      0.46        92
weighted avg       0.55      0.48      0.50        92

              precision    recall  f1-score   support

           1       0.74      0.64      0.68        39
           2       0.65      0.39      0.49        33
           3       0.34      0.65      0.45        20

    accuracy        

In [12]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.74      0.61      0.67       391
         bcc       0.59      0.40      0.48       323
         scc       0.31      0.57      0.41       207

    accuracy                           0.53       921
   macro avg       0.55      0.53      0.52       921
weighted avg       0.59      0.53      0.54       921



# K-Nearest Neighbor

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 25, metric = 'minkowski', p = 1, weights = 'distance')
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=25, p=1,
                     weights='distance')

In [14]:
# Predicting the test set result
y_pred = knn.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[68,  6,  5],
       [ 8, 45,  7],
       [ 8, 30,  8]], dtype=int64)

In [15]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = knn, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.6657904479822289


In [17]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'n_neighbors': [21, 23, 25, 27, 29], 'weights': ['uniform', 'distance'], 'p': [1, 2, 3], 'metric': ['minkowski']},
]

grid_search = GridSearchCV(estimator = knn, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.67
{'metric': 'minkowski', 'n_neighbors': 25, 'p': 1, 'weights': 'distance'}


In [16]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = knn, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.82      0.80      0.81        40
           2       0.63      0.69      0.66        32
           3       0.47      0.43      0.45        21

    accuracy                           0.68        93
   macro avg       0.64      0.64      0.64        93
weighted avg       0.68      0.68      0.68        93

              precision    recall  f1-score   support

           1       0.83      0.74      0.78        39
           2       0.60      0.76      0.67        33
           3       0.40      0.30      0.34        20

    accuracy                           0.65        92
   macro avg       0.61      0.60      0.60        92
weighted avg       0.65      0.65      0.65        92

              precision    recall  f1-score   support

           1       0.77      0.77      0.77        39
           2       0.51      0.64      0.57        33
           3       0.17      0.10      0.12        20

    accuracy        

In [17]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.81      0.77      0.79       391
         bcc       0.57      0.72      0.64       323
         scc       0.40      0.28      0.33       207

    accuracy                           0.64       921
   macro avg       0.59      0.59      0.59       921
weighted avg       0.63      0.64      0.63       921



# Support Vector Machine

In [32]:
# Implementation of Support Vector Machine (SVM)
# Fitting SVM to the training set

from sklearn.svm import SVC
svm = SVC(C = 100, kernel = 'rbf', gamma = 0.001, random_state = 0)
#classifier = SVC(C = 1, kernel = 'poly', degree = 30, random_state = 0)
svm.fit(X_train, y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [33]:
# Predicting the test set result
y_pred = svm.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[72,  4,  3],
       [ 4, 51,  5],
       [ 7, 28, 11]], dtype=int64)

In [34]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = svm, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.7159200296186597


In [31]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'C': [1, 10, 100], 'kernel': ['rbf'], 'gamma': [0.001, 0.01, 0.1, 10]},
]

grid_search = GridSearchCV(estimator = svm, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.72
{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}


In [100]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred, zero_division = 0))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = svm, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.77      0.90      0.83        40
           2       0.48      0.47      0.48        32
           3       0.40      0.29      0.33        21

    accuracy                           0.61        93
   macro avg       0.55      0.55      0.55        93
weighted avg       0.59      0.61      0.60        93

              precision    recall  f1-score   support

           1       0.83      0.77      0.80        39
           2       0.66      0.64      0.65        33
           3       0.42      0.50      0.45        20

    accuracy                           0.66        92
   macro avg       0.64      0.64      0.63        92
weighted avg       0.68      0.66      0.67        92

              precision    recall  f1-score   support

           1       0.75      0.85      0.80        39
           2       0.65      0.61      0.62        33
           3       0.47      0.40      0.43        20

    accuracy        

In [101]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.74      0.83      0.78       391
         bcc       0.58      0.55      0.56       323
         scc       0.41      0.35      0.38       207

    accuracy                           0.62       921
   macro avg       0.58      0.58      0.57       921
weighted avg       0.61      0.62      0.61       921



# Decision Tree

In [37]:
# Some important notes here, before running the Decision Tree Classifier, it's ok to not scale the features, as this model
# does not work using euclidian distance, so it's fine as it is

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10, random_state = 0, splitter = 'random')
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='random')

In [38]:
# Predicting the test set result
y_pred = tree.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[66,  7,  6],
       [ 9, 40, 11],
       [11, 23, 12]], dtype=int64)

In [39]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = tree, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.6072935949648278


In [40]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'criterion': ['entropy', 'gini'], 'splitter': ['best', 'random'], 'ccp_alpha': [0.0, 0.05, 0.1], 'max_depth': [10, 100, 200]}
]

grid_search = GridSearchCV(estimator = tree, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.61
{'ccp_alpha': 0.0, 'criterion': 'entropy', 'max_depth': 10, 'splitter': 'random'}


In [41]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = tree, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.67      0.82      0.74        40
           2       0.50      0.44      0.47        32
           3       0.31      0.24      0.27        21

    accuracy                           0.56        93
   macro avg       0.50      0.50      0.49        93
weighted avg       0.53      0.56      0.54        93

              precision    recall  f1-score   support

           1       0.86      0.77      0.81        39
           2       0.60      0.82      0.69        33
           3       0.33      0.20      0.25        20

    accuracy                           0.66        92
   macro avg       0.60      0.60      0.58        92
weighted avg       0.65      0.66      0.65        92

              precision    recall  f1-score   support

           1       0.64      0.77      0.70        39
           2       0.56      0.61      0.58        33
           3       0.44      0.20      0.28        20

    accuracy        

In [42]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.73      0.75      0.74       391
         bcc       0.56      0.60      0.58       323
         scc       0.32      0.28      0.30       207

    accuracy                           0.59       921
   macro avg       0.54      0.54      0.54       921
weighted avg       0.58      0.59      0.58       921



# Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 200, max_depth = None, criterion = 'entropy', class_weight = 'balanced', random_state = 0)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [44]:
# Predicting the test set result
y_pred = forest.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[75,  3,  1],
       [ 5, 50,  5],
       [ 7, 30,  9]], dtype=int64)

In [45]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = forest, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(np.mean(accuracies))

0.6969640873750463


In [37]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'criterion': ['entropy'], 'max_depth': [None], 'n_estimators': [150, 200, 250], 'class_weight': [None, 'balanced', 'balanced_subsample']},
    {'criterion': ['gini'], 'max_depth': [None], 'n_estimators': [150, 200, 250], 'class_weight': [None, 'balanced', 'balanced_subsample']}
]

grid_search = GridSearchCV(estimator = forest, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.7
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': None, 'n_estimators': 200}


In [46]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = forest, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.80      0.88      0.83        40
           2       0.63      0.75      0.69        32
           3       0.45      0.24      0.31        21

    accuracy                           0.69        93
   macro avg       0.63      0.62      0.61        93
weighted avg       0.66      0.69      0.66        93

              precision    recall  f1-score   support

           1       0.94      0.77      0.85        39
           2       0.63      0.88      0.73        33
           3       0.50      0.35      0.41        20

    accuracy                           0.72        92
   macro avg       0.69      0.67      0.66        92
weighted avg       0.73      0.72      0.71        92

              precision    recall  f1-score   support

           1       0.81      0.87      0.84        39
           2       0.62      0.73      0.67        33
           3       0.45      0.25      0.32        20

    accuracy        

In [47]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.83      0.87      0.85       391
         bcc       0.64      0.75      0.69       323
         scc       0.48      0.30      0.37       207

    accuracy                           0.70       921
   macro avg       0.65      0.64      0.64       921
weighted avg       0.69      0.70      0.69       921



# Artificial Neural Network

In [53]:
# Importing the Keras libraries and packages
import keras
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [93]:
# Create categorical input for predictor

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore')
y_array = Y.values
y_df = y_array.reshape(-1, 1)

y_train = y_train.reshape(-1, 1)
y_test_array = y_test.values
y_test = y_test_array.reshape(-1, 1)

y_use = onehotencoder.fit_transform(y_df).toarray()
y_binary = onehotencoder.fit_transform(y_train).toarray()
y_compare = onehotencoder.fit_transform(y_test).toarray()

In [95]:
# Initializing the ANN
ann = Sequential()

# Adding the input & hidden layer
ann.add(Dense(16, activation = 'relu', input_shape = (12,)))
ann.add(Dense(16, activation = 'relu'))
ann.add(Dense(3, activation = 'softmax'))
ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2),
            ModelCheckpoint(filepath = 'best_model.h5', monitor = 'val_loss', save_best_only = True)]

history = ann.fit(X_train, y_binary, batch_size = 10, epochs = 100, validation_data = (X_test, y_compare))

Train on 736 samples, validate on 185 samples
Epoch 1/100
736/736 [==============================] - 0s 412us/step - loss: 1.2617 - accuracy: 0.2867 - categorical_accuracy: 0.2867 - val_loss: 1.1460 - val_accuracy: 0.3946 - val_categorical_accuracy: 0.3946
Epoch 2/100
736/736 [==============================] - 0s 115us/step - loss: 1.0131 - accuracy: 0.4891 - categorical_accuracy: 0.4891 - val_loss: 1.0040 - val_accuracy: 0.5892 - val_categorical_accuracy: 0.5892
Epoch 3/100
736/736 [==============================] - 0s 115us/step - loss: 0.9126 - accuracy: 0.5883 - categorical_accuracy: 0.5883 - val_loss: 0.9109 - val_accuracy: 0.6378 - val_categorical_accuracy: 0.6378
Epoch 4/100
736/736 [==============================] - 0s 105us/step - loss: 0.8376 - accuracy: 0.6332 - categorical_accuracy: 0.6332 - val_loss: 0.8366 - val_accuracy: 0.6486 - val_categorical_accuracy: 0.6486
Epoch 5/100
736/736 [==============================] - 0s 111us/step - loss: 0.7814 - accuracy: 0.6549 - categ

736/736 [==============================] - 0s 100us/step - loss: 0.5002 - accuracy: 0.7826 - categorical_accuracy: 0.7826 - val_loss: 0.5753 - val_accuracy: 0.7243 - val_categorical_accuracy: 0.7243
Epoch 40/100
736/736 [==============================] - 0s 99us/step - loss: 0.4923 - accuracy: 0.7948 - categorical_accuracy: 0.7948 - val_loss: 0.5551 - val_accuracy: 0.7351 - val_categorical_accuracy: 0.7351
Epoch 41/100
736/736 [==============================] - 0s 99us/step - loss: 0.4936 - accuracy: 0.7772 - categorical_accuracy: 0.7772 - val_loss: 0.5603 - val_accuracy: 0.7405 - val_categorical_accuracy: 0.7405
Epoch 42/100
736/736 [==============================] - 0s 98us/step - loss: 0.4897 - accuracy: 0.7840 - categorical_accuracy: 0.7840 - val_loss: 0.5736 - val_accuracy: 0.7297 - val_categorical_accuracy: 0.7297
Epoch 43/100
736/736 [==============================] - 0s 99us/step - loss: 0.4912 - accuracy: 0.7894 - categorical_accuracy: 0.7894 - val_loss: 0.5605 - val_accuracy:

736/736 [==============================] - 0s 102us/step - loss: 0.4458 - accuracy: 0.8071 - categorical_accuracy: 0.8071 - val_loss: 0.6299 - val_accuracy: 0.7297 - val_categorical_accuracy: 0.7297
Epoch 78/100
736/736 [==============================] - 0s 119us/step - loss: 0.4330 - accuracy: 0.8234 - categorical_accuracy: 0.8234 - val_loss: 0.6056 - val_accuracy: 0.7405 - val_categorical_accuracy: 0.7405
Epoch 79/100
736/736 [==============================] - 0s 117us/step - loss: 0.4335 - accuracy: 0.8207 - categorical_accuracy: 0.8207 - val_loss: 0.6181 - val_accuracy: 0.7405 - val_categorical_accuracy: 0.7405
Epoch 80/100
736/736 [==============================] - 0s 118us/step - loss: 0.4336 - accuracy: 0.8179 - categorical_accuracy: 0.8179 - val_loss: 0.6142 - val_accuracy: 0.7514 - val_categorical_accuracy: 0.7514
Epoch 81/100
736/736 [==============================] - 0s 116us/step - loss: 0.4312 - accuracy: 0.8234 - categorical_accuracy: 0.8234 - val_loss: 0.6239 - val_accur

In [96]:
# Create threshold for multi-predictor
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [97]:
# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_compare.argmax(axis=1), y_pred.argmax(axis=1))

cm

array([[73,  2,  4],
       [ 5, 45, 10],
       [ 7, 20, 19]], dtype=int64)

In [98]:
# Calculating the accuracy and other metrics
#, scoring=make_scorer(classification_report_with_accuracy_score)

from sklearn.model_selection import StratifiedKFold, cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Build the Model
def buildmodel():
    ann = Sequential()
    ann.add(Dense(16, activation = 'relu', input_shape = (12,)))
    ann.add(Dense(16, activation = 'relu'))
    ann.add(Dense(3, activation = 'softmax'))
    ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])
    return(ann)


#Make our customer score
def classification_report_with_accuracy_score(y_use, y_pred):
    originalclass.extend(y_use)
    predictedclass.extend(y_pred.argmax(axis=1))
    print(classification_report(y_use, y_pred.argmax(axis=1)))
    return accuracy_score(y_use, y_pred.argmax(axis=1)) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
ann = KerasRegressor(build_fn=buildmodel, epochs = 100, batch_size = 10, verbose = 0)
nested_score = cross_val_score(estimator = ann, X = X, y = y_use.argmax(1), cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88        40
           1       0.80      0.62      0.70        32
           2       0.46      0.57      0.51        21

    accuracy                           0.73        93
   macro avg       0.71      0.70      0.70        93
weighted avg       0.75      0.73      0.73        93

              precision    recall  f1-score   support

           0       1.00      0.67      0.80        39
           1       0.87      0.39      0.54        33
           2       0.35      0.90      0.51        20

    accuracy                           0.62        92
   macro avg       0.74      0.65      0.62        92
weighted avg       0.81      0.62      0.64        92

              precision    recall  f1-score   support

           0       0.89      0.79      0.84        39
           1       0.67      0.36      0.47        33
           2       0.33      0.65      0.44        20

    accuracy        

In [99]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.86      0.82      0.84       391
         bcc       0.67      0.66      0.67       323
         scc       0.40      0.45      0.42       207

    accuracy                           0.68       921
   macro avg       0.65      0.64      0.64       921
weighted avg       0.69      0.68      0.69       921



# XGBoost

In [48]:
# Importing the XGB Library and fit it to the data
from xgboost import XGBClassifier
xgb = XGBClassifier(objective = 'multi:softmax', booster = 'dart', n_estimators = 200, normalize_type = 'forest', rate_drop = 0.2, sample_type = 'uniform', skip_drop = 0)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              normalize_type='forest', nthread=None, objective='multi:softprob',
              random_state=0, rate_drop=0.2, reg_alpha=0, reg_lambda=1,
              sample_type='uniform', scale_pos_weight=1, seed=None, silent=None,
              skip_drop=0, subsample=1, verbosity=1)

In [49]:
# Predicting the test set result
y_pred = xgb.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[74,  3,  2],
       [ 6, 49,  5],
       [ 9, 28,  9]], dtype=int64)

In [50]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(np.mean(accuracies))

0.6806738245094409


In [12]:
# Applying Grid Search to Find the Best Hyperparameter
# {'objective': ['multi:softmax'], 'n_estimators': [25, 50, 100], 'booster': ['gbtree'], 'eta': [0.001, 0.002, 0.004], 'max_depth':[3], 'gamma': [0], 'subsample': [1], 'sampling_method': ['uniform', 'gradient_based']},
# {'objective': ['multi:softmax'], 'n_estimators': [109], 'booster': ['dart'], 'sample_type': ['uniform', 'weighted'], 'normalize_type': ['tree', 'forest'], 'rate_drop': [0, 0.1, 0.2], 'skip_drop': [0, 0.1, 0.2]}
# {'objective': ['multi:softmax'], 'n_estimators': [10, 25, 50], 'booster': ['gblinear'], 'feature_selector': ['greedy', 'cyclic', 'random', 'shuffle', 'thrifty'], 'updater': ['shotgun', 'coord_descent'], 'lambda': [0, 0.5, 1], 'alpha': [0, 0.5, 1]}

from sklearn.model_selection import GridSearchCV
parameters = [
    {'objective': ['multi:softmax'], 'n_estimators': [25, 50, 100], 'booster': ['gbtree'], 'eta': [0.001, 0.002, 0.004], 'max_depth':[3], 'gamma': [0], 'subsample': [1], 'sampling_method': ['uniform', 'gradient_based']},
    {'objective': ['multi:softmax'], 'n_estimators': [25, 50, 100], 'booster': ['dart'], 'sample_type': ['uniform', 'weighted'], 'normalize_type': ['tree', 'forest'], 'rate_drop': [0, 0.1, 0.2], 'skip_drop': [0, 0.1, 0.2]}
]

grid_search = GridSearchCV(estimator = xgb, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.7
{'booster': 'dart', 'n_estimators': 100, 'normalize_type': 'forest', 'objective': 'multi:softmax', 'rate_drop': 0.2, 'sample_type': 'uniform', 'skip_drop': 0}


In [51]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = xgb, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.80      0.88      0.83        40
           2       0.62      0.62      0.62        32
           3       0.35      0.29      0.32        21

    accuracy                           0.66        93
   macro avg       0.59      0.60      0.59        93
weighted avg       0.64      0.66      0.64        93

              precision    recall  f1-score   support

           1       0.89      0.82      0.85        39
           2       0.69      0.88      0.77        33
           3       0.43      0.30      0.35        20

    accuracy                           0.73        92
   macro avg       0.67      0.67      0.66        92
weighted avg       0.72      0.73      0.72        92

              precision    recall  f1-score   support

           1       0.82      0.79      0.81        39
           2       0.62      0.70      0.66        33
           3       0.41      0.35      0.38        20

    accuracy        

In [52]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.83      0.88      0.86       391
         bcc       0.65      0.74      0.69       323
         scc       0.45      0.31      0.37       207

    accuracy                           0.70       921
   macro avg       0.65      0.64      0.64       921
weighted avg       0.69      0.70      0.69       921

